### plot strike, bid/ask, call, put amount v.s. PERP and underlying
   * using different marker size to represent amount of bid/ask
   * using diffrernt color to mark bid and ask 
   * left y-axis for strike
   * right y-axis for underlying price
   * hover mouse to show option price
   * 5 expire x put/call x bid/ask = 20 plots => 20 html files

In [ ]:
from cdcqr.common.config import LOCAL_DATA_DIR
from cdcqr.data.dataloader import data_loader
from cdcqr.data.deribit.data_utils import DeribitUtils
import pandas as pd
from datetime import datetime
import plotly.express as px
import os
from cdcqr.common.config import LOCAL_FIGURE_DIR

In [ ]:
index_1m = pd.date_range(start=datetime(2021,5,19,0,0), end=datetime(2021,5,19,23,59), freq='1Min')

#### get quote data on 20210519

In [ ]:
exchange = 'deribit'
# date = '2021-05-19'
date = '2021-10-29'
symbol = 'OPTIONS'
data_type = 'quotes'
opt_quote = data_loader(exchange, date, data_type,symbol)

In [ ]:
#### get PERP data on 20210519
symbol = 'BTC-PERPETUAL'
PERP_quote_btc = data_loader(exchange, date,data_type,symbol)

In [ ]:
PERP_quote_btc['dt'] = pd.to_datetime(PERP_quote_btc['timestamp'], unit='us')
PERP_quote_btc['BTC_PERP_mid_price'] = (PERP_quote_btc['ask_price']+PERP_quote_btc['bid_price'])/2

In [ ]:
PERP_quote_btc_1 = PERP_quote_btc.set_index('dt')['BTC_PERP_mid_price']
PERP_quote_btc_2 = PERP_quote_btc_1[~PERP_quote_btc_1.index.duplicated(keep='last')]
PERP_quote_btc_3 = PERP_quote_btc_2.reindex(index_1m, method='ffill')

In [ ]:
#### get optchain data on 20210519
optchain1Min_20210519_20211231 = pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain1Min_20210519_20211231.pkl'))
optchain1Min_20210519_20211231['mid_vol'] = 0.5*(optchain1Min_20210519_20211231['aiv'] + optchain1Min_20210519_20211231['biv'])

In [ ]:
optchain1Min_20210519_exp_i = optchain1Min_20210519_20211231.copy()
optchain1Min_20210519_exp_i_reduced = optchain1Min_20210519_exp_i[['symbol','strike','underly','s','aiv','biv']]
optchain1Min_20210519_exp_i_reduced = optchain1Min_20210519_exp_i_reduced[optchain1Min_20210519_exp_i_reduced['symbol'].str.contains('BTC-')]
optchain1Min_20210519_exp_i_reduced = optchain1Min_20210519_exp_i_reduced.pipe(DeribitUtils.parse_optSymbol_col)

In [ ]:
# 1. get underlying price
optchain1Min_20210519_exp_i_reduced_1 = optchain1Min_20210519_exp_i_reduced['s']
optchain1Min_20210519_exp_i_reduced_2 =  optchain1Min_20210519_exp_i_reduced_1[~optchain1Min_20210519_exp_i_reduced_1.index.duplicated(keep='last')]
optchain1Min_20210519_exp_i_reduced_2 = optchain1Min_20210519_exp_i_reduced_2.sort_index()
optchain1Min_20210519_exp_i_reduced_3 = optchain1Min_20210519_exp_i_reduced_2.reindex(index_1m, method='ffill').to_frame()

In [ ]:
# 2. get ask iv per strike, per type
optchain1Min_20210519_exp_i_reduced_1a = optchain1Min_20210519_exp_i_reduced[['aiv','biv','strike','type']]

In [ ]:
optchain1Min_20210519_exp_i_reduced_2a = optchain1Min_20210519_exp_i_reduced_1a.reset_index().rename(columns={'tm':'index'})
optchain1Min_20210519_exp_i_reduced_2a.head(2)

In [ ]:
optchain1Min_20210519_exp_i_reduced_2a['strike'].unique()

In [ ]:
optchain1Min_20210519_exp_i_reduced_3.columns = ['underlying_price']
df_underlying = optchain1Min_20210519_exp_i_reduced_3.join(PERP_quote_btc_3).reset_index()

In [ ]:
df_underlying.head(2)

In [ ]:
# processing quote data
# only look at BTC
opt_quote_btc = opt_quote_parsed.query('instrument=="BTC"')

In [ ]:
# select relevant expire dates
selected_expr_dates = DeribitUtils.get_important_option_expire_dates(pd.to_datetime(date))
selected_expr_dates

In [ ]:
opt_quote_btc['exp_date'] = opt_quote_btc['expire'].dt.date

In [ ]:
expire_i = selected_expr_dates[-1]
opt_quote_btc_expire_i_c = opt_quote_btc.query('exp_date==@expire_i & type =="C"')
opt_quote_btc_expire_i_p = opt_quote_btc.query('exp_date==@expire_i & type =="P"')

In [ ]:
list(sorted(opt_quote_btc_expire_i_c['strike'].unique())) == sorted(opt_quote_btc_expire_i_p['strike'].unique())
strike_list = list(sorted(opt_quote_btc_expire_i_c['strike'].unique()))
len(strike_list)

In [ ]:
opt_quote_btc_expire_i_c_reduced = opt_quote_btc_expire_i_c.drop(labels=['exchange','symbol','timestamp','local_timestamp','type','expire','t2m'], axis=1)
opt_quote_btc_expire_i_p_reduced = opt_quote_btc_expire_i_p.drop(labels=['exchange','symbol','timestamp','local_timestamp','type','expire','t2m'], axis=1)

In [ ]:
opt_quote_btc_expire_i_c_reduced_k = opt_quote_btc_expire_i_c_reduced.query('strike==90000').set_index('timestamp_dt')
opt_quote_btc_expire_i_p_reduced_k = opt_quote_btc_expire_i_p_reduced.query('strike==90000').set_index('timestamp_dt')

In [ ]:
opt_quote_btc_expire_i_c_reduced_k_1 = opt_quote_btc_expire_i_c_reduced_k[~opt_quote_btc_expire_i_c_reduced_k.index.duplicated(keep='last')]
opt_quote_btc_expire_i_p_reduced_k_1 = opt_quote_btc_expire_i_p_reduced_k[~opt_quote_btc_expire_i_p_reduced_k.index.duplicated(keep='last')]

In [ ]:
strike2df = {}
for strike in opt_quote_btc_expire_i_c_reduced['strike'].unique():
    #print(strike)
    opt_quote_btc_expire_i_c_reduced_k_1 = opt_quote_btc_expire_i_c_reduced.query('strike==@strike').set_index('timestamp_dt')
    opt_quote_btc_expire_i_c_reduced_k_1 = opt_quote_btc_expire_i_c_reduced_k_1[~opt_quote_btc_expire_i_c_reduced_k_1.index.duplicated(keep='last')]
    opt_quote_btc_expire_i_c_reduced_k_1_1min = opt_quote_btc_expire_i_c_reduced_k_1.reindex(index_1m, method='ffill')
    strike2df[strike] = opt_quote_btc_expire_i_c_reduced_k_1_1min
    #display(opt_quote_btc_expire_i_c_reduced_k_1_1min.head(2))
df_combined = pd.concat(strike2df.values()).reset_index()
df_combined['ask_amount'] = df_combined['ask_amount'].fillna(0)
df_combined['bid_amount'] = df_combined['bid_amount'].fillna(0)
df_combined_c = df_combined.copy()

In [ ]:
strike2df = {}
for strike in opt_quote_btc_expire_i_p_reduced['strike'].unique():
    #print(strike)
    opt_quote_btc_expire_i_p_reduced_k_1 = opt_quote_btc_expire_i_p_reduced.query('strike==@strike').set_index('timestamp_dt')
    opt_quote_btc_expire_i_p_reduced_k_1 = opt_quote_btc_expire_i_p_reduced_k_1[~opt_quote_btc_expire_i_p_reduced_k_1.index.duplicated(keep='last')]
    opt_quote_btc_expire_i_p_reduced_k_1_1min = opt_quote_btc_expire_i_p_reduced_k_1.reindex(index_1m, method='ffill')
    strike2df[strike] = opt_quote_btc_expire_i_p_reduced_k_1_1min
    #display(opt_quote_btc_expire_i_p_reduced_k_1_1min.head(2))
df_combined = pd.concat(strike2df.values()).reset_index()
df_combined['ask_amount'] = df_combined['ask_amount'].fillna(0)
df_combined['bid_amount'] = df_combined['bid_amount'].fillna(0)
df_combined_p = df_combined.copy()

In [ ]:
df_combined_c['type'] = 'C'
df_combined_p['type'] = 'P'
df_combined = df_combined_c.append(df_combined_p)

In [ ]:
df_combined['strike_shifted'] = df_combined['strike'] + 100*((df_combined['type']=='C').astype(int)-0.5)*2

In [ ]:
df_combined.head()

In [ ]:
from plotly.subplots import make_subplots
subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig1a = px.scatter(df_combined, x="index", y="strike", size="ask_amount",  hover_name="ask_price", color="strike", title='expire date',width=1500, height=1200)
fig1b = px.scatter(df_combined, x="index", y="strike", size="bid_amount",  hover_name="bid_price", color="strike", title='expire date',width=1500, height=1200)

In [ ]:
optchain1Min_20210519_exp_i_reduced_1a.head()

In [ ]:
# remove duplicates in opt chain
optchain1Min_20210519_exp_i_reduced_3a = optchain1Min_20210519_exp_i_reduced_2a[~optchain1Min_20210519_exp_i_reduced_2a.set_index(['type','strike','index']).index.duplicated(keep='last')]

In [ ]:
df_combined_ext = pd.merge(left=df_combined, right = optchain1Min_20210519_exp_i_reduced_3a, on=['index','strike','type'], how='left')

In [ ]:
df_combined_ext['mvol'] = 0.5*(df_combined_ext['aiv']+df_combined_ext['biv'])

In [ ]:
display(df_combined.shape)
display(df_combined_ext.shape)

In [ ]:
df_combined_ext['total_amount'] = df_combined_ext['bid_amount'] + df_combined_ext['ask_amount']

In [ ]:
fig1 = px.scatter(df_combined_ext, x="index", y="strike_shifted", size="ask_amount",  hover_data=['strike','ask_price','ask_amount','type', 'biv'], color="biv", title='',width=1500, height=1200)
fig1b = px.scatter(df_combined_ext, x="index", y="strike_shifted", size="bid_amount",  hover_data=['strike','bid_price','bid_amount','type', 'biv'], color="biv", title='',width=1500, height=1200)
fig1c = px.scatter(df_combined_ext, x="index", y="strike_shifted", size="total_amount", hover_data=['strike','ask_price','ask_amount','bid_price','bid_amount','type', 'aiv','biv','mvol'], color="mvol", title='',width=1500, height=1200)

In [ ]:
fig2 = px.line(df_underlying, x="index", y=['underlying_price','BTC_PERP_mid_price'], width=1500, height=1200)

In [ ]:
fig3 = go.Figure(data=fig1.data + fig2.data,)
fig3.update_layout(
    autosize=False,
    width=1500,
    height=1200,
    title='BTC ask price and amount v.s. underlying & PERP')
fig3.show()
fig3.write_html(os.path.join(LOCAL_FIGURE_DIR,'option_book', '202105', "BTC_options_20210519_20211231_ask.html"))

In [ ]:
fig3b = go.Figure(data=fig1b.data + fig2.data,)
fig3b.update_layout(
    autosize=False,
    width=1500,
    height=1200,
    title='BTC bid price and amount v.s. underlying & PERP')
fig3b.show()
fig3b.write_html(os.path.join(LOCAL_FIGURE_DIR,'option_book', '202105', "BTC_options_20210519_20211231_bid.html"))

In [ ]:
fig3c = go.Figure(data=fig1c.data + fig2.data,)
fig3c.update_layout(
    autosize=False,
    width=1500,
    height=1200,
    title='BTC price and total amount v.s. underlying & PERP')
fig3c.show()
fig3c.write_html(os.path.join(LOCAL_FIGURE_DIR, 'option_book', '202105', "BTC_options_20210519_20211231_all_quotes.html"))

### plot 20211029 data



In [ ]:
%store -r df_combined_ext_20211029
%store -r df_underlying_20211029

In [ ]:
df_combined_ext_20211029

In [ ]:
df_underlying_20211029

In [ ]:
subfig = make_subplots(specs=[[{"secondary_y": True}]])
fig1a = px.scatter(df_combined_ext_20211029, x="index", y="strike_shifted", size="ask_amount",  hover_data=['strike','ask_price','ask_amount','type', 'aiv'], color="aiv", title='',width=1500, height=1200)
fig1b = px.scatter(df_combined_ext_20211029, x="index", y="strike_shifted", size="bid_amount",  hover_data=['strike','bid_price','bid_amount','type', 'biv'], color="biv", title='',width=1500, height=1200)
fig1c = px.scatter(df_combined_ext_20211029, x="index", y="strike_shifted", size="total_amount", hover_data=['strike','ask_price','ask_amount','bid_price','bid_amount','type', 'aiv','biv','mvol'], color="mvol", title='',width=1500, height=1200)

In [ ]:
fig2 = px.line(df_underlying_20211029, x="index", y=['underlying_price','PERP_mid_price'], width=1500, height=1200)

In [ ]:
fig3a = go.Figure(data=fig1a.data + fig2.data,)
fig3a.update_layout(
    autosize=False,
    width=1500,
    height=1200,
    title='BTC ask price and amount v.s. underlying & PERP')
#fig3a.show()
fig3a.write_html(os.path.join(LOCAL_FIGURE_DIR,'option_book', '202110', "BTC_options_20211029_20220624_ask.html"))

In [ ]:
fig = go.Figure(data=fig1b.data + fig2.data)
fig.update_layout(
    autosize=False,
    width=1500,
    height=1200,
    title='BTC ask price and amount v.s. underlying & PERP')
#fig3b.show()
fig.write_html(os.path.join(LOCAL_FIGURE_DIR,'option_book', '202110', "BTC_options_20211029_20220624_bid.html"))

In [ ]:
fig = go.Figure(data=fig1c.data + fig2.data,)
fig.update_layout(
    autosize=False,
    width=1500,
    height=1200,
    title='BTC ask price and amount v.s. underlying & PERP')
fig.write_html(os.path.join(LOCAL_FIGURE_DIR,'option_book', '202110', "BTC_options_20211029_20220624_all_quotes.html"))